# Imports

In [1]:
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))
from helpers_df import create_lag_df

# Read Data

This data set takes into account passing data from 2012 - 2022. This data is unfiltered.

In [90]:
df_roster = pd.read_pickle("../../interactive/df_roster_2012_2022.pkl")

In [91]:
df_roster_seasonal = df_roster.groupby(["position", "player_name", "player_id"]).agg({"age": ["mean"]})

In [92]:
df_roster_seasonal.columns = list(map("_".join, df_roster_seasonal.columns))
df_roster_seasonal.reset_index(inplace=True)

In [93]:
df_seasonal = pd.read_pickle("../../interactive/df_seasonal_2012_2022.pkl")

In [94]:
df_merge = df_seasonal.merge(df_roster_seasonal, on="player_id", how="left")

# Create Data Frame for Runs

In [95]:
df_rbs = df_merge.copy().query("season_type == 'REG' and position == 'RB'").drop_duplicates().dropna()

In [96]:
df_rbs["total_yards"] = df_rbs["rushing_yards"] + df_rbs["receiving_yards"]
df_rbs["total_tds"] = df_rbs["rushing_tds"] + df_rbs["receiving_tds"]
df_rbs["touches"] = df_rbs["targets"] + df_rbs["carries"]
df_rbs["total_first_downs"] = df_rbs["rushing_first_downs"] + df_rbs["receiving_first_downs"]

In [97]:
df_rbs.drop(columns=['completions', 'attempts',
       'passing_yards', 'passing_tds', 'interceptions', 'sacks', 'sack_yards',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'dakota', 'special_teams_tds'], inplace=True)

# Create Lagged Version

In [98]:
df_lag = create_lag_df(df_rbs)

# Calculating Percent Fantasy Points

In [99]:
query = "games >= 10 and total_yards >= 800"

df_rbs = df_rbs.query(query).copy()

In [100]:
df_rbs["fantasy_points_ppr"] = df_rbs["fantasy_points_ppr"].replace(0, np.nan)

In [101]:
df_rbs["fantasy_points_ppr_percentage_rushing"] = (df_rbs["rushing_tds"] * 6
    + df_rbs["rushing_fumbles_lost"] * -2 + df_rbs["rushing_2pt_conversions"] * 2
    + df_rbs["rushing_yards"] / 10) / df_rbs["fantasy_points_ppr"]

df_rbs["fantasy_points_ppr_percentage_receiving"] = 1 - df_rbs["fantasy_points_ppr_percentage_rushing"]

In [102]:
df_rbs["fantasy_points_ppr_percentage_rushing"].mean()

0.618953805586603

In [103]:
df_rbs["age_mean"] = df_rbs["age_mean"].astype(int)

In [104]:
df_rbs_percent_by_age = df_rbs \
    .groupby(["age_mean"]) \
    .agg({
        "fantasy_points_ppr_percentage_rushing": ["mean"],
        "fantasy_points_ppr_percentage_receiving": ["mean"]
    })

In [105]:
df_rbs_percent_by_age.columns = list(map("_".join, df_rbs_percent_by_age.columns))
df_rbs_percent_by_age.reset_index(inplace=True)

In [106]:
rename_dict = {
    'age_mean': "Age",
    'fantasy_points_ppr_percentage_rushing_mean': "Percent Rushing Work",
    'fantasy_points_ppr_percentage_receiving_mean': "Percent Receiving Work",
}

In [107]:
df_rbs_percent_by_age.rename(columns=rename_dict, inplace=True)

In [108]:
df_rbs_percent_by_age["Percent Receiving Work"].mean()

0.380814340550596

In [109]:
df_rbs_percent_by_age

,Age,Percent Rushing Work,Percent Receiving Work
0,21,0.666629,0.333371
1,22,0.665691,0.334309
2,23,0.634846,0.365154
3,24,0.580673,0.419327
4,25,0.615374,0.384626
5,26,0.548903,0.451097
6,27,0.727092,0.272908
7,28,0.628546,0.371454
8,29,0.492776,0.507224
9,30,0.699007,0.300993


# Graphing Fantasy Percentages

In [111]:
fig = px.bar(df_rbs_percent_by_age, x='Age',
              y=['Percent Rushing Work', 'Percent Receiving Work'],
             )

# Update layout
fig.update_layout(
    title='Distribution of Fantasy Points vs. Age (10 Games and 800 Yards)',
    xaxis_title='Age',
    yaxis_title='Percentage Fantasy Points',
    legend_title='Key'
)

# save the figure
fig.write_html("../../interactive/RB/stability-total/season/distribution-of-fantasy-points-10-games-800-yards.html")

# Show the figure
fig.show()